In [1]:
!pip install -q datasets transformers==4.31.0 trl==0.4.7 accelerate==0.21.0 peft==0.4.0 scipy wandb bitsandbytes==0.40.2 sentencepiece


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

import wandb
from getpass import getpass
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datasets import load_dataset

In [3]:
model_name = 'openlm-research/open_llama_3b_v2'
dataset_name = 'tatsu-lab/alpaca'


In [4]:
dataset = load_dataset(dataset_name)['train'].train_test_split(test_size = 0.2)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
train_dataset, eval_dataset = dataset['train'], dataset['test']

In [6]:
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

lora_config = LoraConfig(
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    r = lora_r,
    bias = 'none',
    task_type = 'CAUSAL_LM'
)

In [7]:
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

bnb_config = BitsAndBytesConfig(
    load_in_4bit = use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_use_double_quant = use_nested_quant,
)

In [8]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto'
)

In [9]:
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = False)
tokenizer.pad_token = tokenizer.eos_token

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [19]:
output_dir = "./results"
num_train_epochs = -1
fp16 = False
bf16 = False
per_device_train_batch_size = 16
per_device_eval_batch_size = 16
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 3e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = 5000
warmup_ratio = 0.03
group_by_length = True
save_steps = 100
logging_steps = 25
max_seq_length = None
packing = False

In [20]:
arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size= per_device_eval_batch_size,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=200,  # Evaluate every 20 steps
    save_strategy = 'steps',
    save_safetensors = True
)

In [21]:
trainer = SFTTrainer(
    model = base_model,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    peft_config = lora_config,
    dataset_text_field = 'text',
    tokenizer = tokenizer,
    args = arguments
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [22]:
trainer.train()

Step,Training Loss,Validation Loss
200,0.804200,1.040389
400,0.794600,1.017954
600,0.786200,1.007280
800,0.766000,1.005560
1000,0.761000,0.998650
1200,0.766300,0.992431
1400,0.764600,0.989242
1600,0.757500,0.984832
1800,0.753500,0.984409


KeyboardInterrupt: 

In [ ]:
!rm -r ./food_b